In [3]:
from os import listdir
from os.path import isdir, join
from tensorflow.keras import layers, models
import numpy as np

In [4]:
dataset_path = 'C:/Users/USER/Downloads/Compressed/speech_commands_v0.02'
all_targets = [name for name in listdir(dataset_path) if isdir(join(dataset_path, name))]
print(all_targets)

['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero', '_background_noise_']


In [5]:
commands = [all_targets[i] for i in [0,9,15,19,20,22,26,-1]]
commands

['backward',
 'forward',
 'left',
 'off',
 'on',
 'right',
 'stop',
 '_background_noise_']

In [7]:
# Settings
feature_sets_filename = 'commands_mfcc_sets.npz'
model_filename ='command_detection_model.h5'
labels model_filename= ['backward','forward','left','off', 'on','right','stop','silence']

In [8]:
feature_sets = np.load(feature_sets_filename)
print(feature_sets.files)

['x_train', 'y_train', 'x_val', 'y_val', 'x_test', 'y_test']


In [9]:
x_train = feature_sets['x_train']
y_train = feature_sets['y_train']
x_val = feature_sets['x_val']
y_val = feature_sets['y_val']
x_test = feature_sets['x_test']
y_test = feature_sets['y_test']

In [10]:
# CNN for TF expects (batch, height, width, channels)
# So we reshape the input tensors with a "color" channel of 1
x_train = x_train.reshape(x_train.shape[0], 
                          x_train.shape[1], 
                          x_train.shape[2], 
                          1)
x_val = x_val.reshape(x_val.shape[0], 
                      x_val.shape[1], 
                      x_val.shape[2], 
                      1)
x_test = x_test.reshape(x_test.shape[0], 
                        x_test.shape[1], 
                        x_test.shape[2], 
                        1)
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(16835, 16, 16, 1)
(2110, 16, 16, 1)
(2118, 16, 16, 1)


In [11]:
sample_shape = x_train.shape[1:]
print(sample_shape)

(16, 16, 1)


In [12]:
model = models.Sequential()
model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=sample_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Classifier
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [13]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [14]:
history = model.fit(x_train, 
                    y_train, 
                    epochs=50, 
                    batch_size=100, 
                    validation_data=(x_val, y_val))

Train on 16835 samples, validate on 2110 samples
Epoch 1/50
16835/16835 [==============================] - 3s 158us/sample - loss: 1.6682 - acc: 0.3499 - val_loss: 1.0264 - val_acc: 0.6540
Epoch 2/50
16835/16835 [==============================] - 2s 118us/sample - loss: 1.0279 - acc: 0.6228 - val_loss: 0.7379 - val_acc: 0.7318
Epoch 3/50
16835/16835 [==============================] - 2s 118us/sample - loss: 0.8361 - acc: 0.7018 - val_loss: 0.6231 - val_acc: 0.7754
Epoch 4/50
16835/16835 [==============================] - 2s 119us/sample - loss: 0.7296 - acc: 0.7463 - val_loss: 0.5200 - val_acc: 0.8166
Epoch 5/50
16835/16835 [==============================] - 2s 126us/sample - loss: 0.6640 - acc: 0.7743 - val_loss: 0.4772 - val_acc: 0.8412
Epoch 6/50
16835/16835 [==============================] - 2s 124us/sample - loss: 0.6080 - acc: 0.7960 - val_loss: 0.4570 - val_acc: 0.8445
Epoch 7/50
16835/16835 [==============================] - 2s 126us/sample - loss: 0.5710 - acc: 0.8040 - val_lo

In [15]:
model.evaluate(x_test, y_test)

2118/2118 [==============================] - 0s 55us/sample - loss: 0.3887 - acc: 0.9008


[0.3887217147407721, 0.9008499]

In [16]:
model_filename

'command_detection_model.h5'

In [17]:
model.save(model_filename)

In [22]:
from tensorflow import lite

In [24]:
keras_model_filename="C:/Users/USER/Desktop/snow_bois/Commands_feature_extraction/command_detection_model.h5"
filename="model.tflite"

In [25]:
model=models.load_model(keras_model_filename)
converter=lite.TFLiteConverter.from_keras_model(model)
tflite=converter.convert()
open (filename,"wb").write(tflite)

AttributeError: 'str' object has no attribute 'decode'